使用特定初始化是让反向传播时，传播前和传播后，某层方差保持不变，从而缓解梯度爆炸和梯度消失的问题

普通的sigmoid函数的平均值为0.5并非0，导致神经元输出的方差将大于输入的方差


In [14]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


In [15]:
# Glorot 和 He 初始化
[name for name in dir(keras.initializers) if not name.startswith('_')]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

|初始化|激活函数|$\sigma^2$|
|-|-|-|
|Glorot|None,tanh,logistic,softmax|$\frac{1}{fan_{avg}}$|
|He|ReLU和变体|$\frac{2}{fan_{in}}$|
|LeCun|SELU|$\frac{1}{fan_{in}}$|

In [16]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [17]:
# 如果使用均匀分布但基于fan_avg而不是fan_in
# 进行He初始化，则可以使用Variance_Scaling初始化
init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg'
                                , distribution='uniform')
        
keras.layers.Dense(10, activation='relu', kernel_initializer=init)

相比于ReLU，leaky ReLU能够防止加权和为负数时，神经元只会输出0导致神经元死亡
$$ReLU(z)=\max(0,z) $$

$$leakyReLU(z)=\max(\alpha z,z) $$

后续的，还有随机泄露ReLU（RReLU），在训练过程中在给定范围内随机选择 $\alpha $

有参数化泄露ReLU（PReLU）， $\alpha $将在训练期间学习而不作为超参数

还有一种激活函数，指数线性单位（Exponential Linear Unit，ELU）

$$
ELU(z)=
\begin{cases}

\alpha(e^z - 1)\quad if\quad z<0\\
z\quad if\quad z\geq 0


\end{cases}
$$
如果 $\alpha=1 $则函数所有位置都是平滑的，有助于梯度加速下降

ELU的主要缺点时它的计算比ReLU及变体要慢

SELU 可以使得网路是自归一化的，但是有条件p298

In [18]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [19]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [20]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [21]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 7ms/step - loss: 1.2936 - accuracy: 0.6064 - val_loss: 0.8771 - val_accuracy: 0.7152
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.7899 - accuracy: 0.7424 - val_loss: 0.7055 - val_accuracy: 0.7706
Epoch 3/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.6771 - accuracy: 0.7810 - val_loss: 0.6373 - val_accuracy: 0.7912
Epoch 4/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.6178 - accuracy: 0.7972 - val_loss: 0.5827 - val_accuracy: 0.8092
Epoch 5/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5797 - accuracy: 0.8079 - val_loss: 0.5511 - val_accuracy: 0.8148
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5524 - accuracy: 0.8154 - val_loss: 0.5280 - val_accuracy: 0.8264
Epoch 7/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5317 - accuracy: 0.8205 - val_loss: 0.5097 - val_ac

* Batch Normaliztion批量归一化
  
    更换激活函数成功缓解了训练一开始时的梯度爆炸、消失，而无法保证训练一段时间后的旧疾复发

    批量归一化通过让模型学习各层输入的最佳缩放和均值，从而缓解训练一段时间后的梯度爆炸、消失
* 大致方法

    在隐藏层的激活函数前，通过该层输入的均值和标准差的 **移动平均值**来估计训练期间的最终统计信息

    1. 通过反向传播想学习输出缩放向量$\gamma $和输出偏移向量$\beta $
    2. 学习使用指数移动平均值估计的最终的输入均值向量$\mu $和最终输入标准差向量$\sigma $
* 具体实现

    使用API实现非常简单直观，只需在每个隐藏层的激活函数之前或之后添加一个批量归一化的层

    但是BN论文的作者主张在激活函数之前添加批量归一化层

    首先是在激活后使用BN，随后是在激活前使用BN

In [22]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # 可选地在模型的第一层后添加一个BN
    keras.layers.BatchNormalization(), 
    # 在激活函数后添加BN 
    keras.layers.Dense(300, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_13 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_14 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [24]:
# 每个BN层的每个输入添加了四个参数
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))


现在试试在激活之前使用BN层

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # 可选地在模型的第一层之后添加一个BN层
    keras.layers.BatchNormalization(), 
    # 在激活函数之前使用BN， 
    # 在BN层之前的层不需要偏置项，因为BN自带偏置项
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

在反向传播期间裁剪梯度，让它们永远不会超过某个阈值，是另一个缓解梯度爆炸的流行技术

keras中仅仅需要设置clipvalue或clipnorm参数，即可实现梯度裁剪

该优化器会将梯度向量的每个分量都蔡建伟-1.0到1.0之间的值，即将所有损失的偏导数限制在这个范围之间，阈值是可以调整的超参数

应当通过设置clipnorm而不是clipvalue按照范数裁剪，确保梯度裁剪不会改变梯度的方向（按值裁剪、按范数裁剪）

In [ ]:
from pickletools import optimize


optimizer = keras.optimizers.SGD(clipvalue=1.0)
optimizer = keras.optimizers.SGD(clipnorm=1.0)